In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("C:/Users/navir/Downloads/clean_data_after_eda.csv")

In [4]:
df["date_activ"] = pd.to_datetime(df["date_activ"], format='%Y-%m-%d')
df["date_end"] = pd.to_datetime(df["date_end"], format='%Y-%m-%d')
df["date_modif_prod"] = pd.to_datetime(df["date_modif_prod"], format='%Y-%m-%d')
df["date_renewal"] = pd.to_datetime(df["date_renewal"], format='%Y-%m-%d')

In [5]:
df.head(3)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.000131,4.100838e-05,0.000908,2.086294,99.530517,44.235794,2.086425,9.953056e+01,44.236702,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000003,1.217891e-03,0.000000,0.009482,0.000000,0.000000,0.009485,1.217891e-03,0.000000,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000004,9.450150e-08,0.000000,0.000000,0.000000,0.000000,0.000004,9.450150e-08,0.000000,0


3. Feature engineering
Difference between off-peak prices in December and preceding January

Below is the code created by your colleague to calculate the feature described above. Use this code to re-create this feature and then think about ways to build on this feature to create features with a higher predictive power.

In [7]:
price_df = pd.read_csv("C:/Users/navir/Downloads/price_data (1).csv")
price_df["price_date"] = pd.to_datetime(price_df["price_date"], format='%Y-%m-%d')
price_df.head()

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0


In [12]:
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()

# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff = pd.merge(dec_prices.rename(columns={'price_off_peak_var': 'dec_1', 'price_off_peak_fix': 'dec_2'}), jan_prices.drop(columns='price_date'), on='id')


In [13]:
diff

,id,price_date,dec_1,dec_2,price_off_peak_var,price_off_peak_fix
0,0002203ffbb812588b632b9e628cc38d,2015-12-01,0.119906,40.728885,0.126098,40.565969
1,0004351ebdd665e6ee664792efc4fd13,2015-12-01,0.143943,44.444710,0.148047,44.266931
2,0010bcc39e42b3c2131ed2ce55246e3c,2015-12-01,0.201280,45.944710,0.150837,44.444710
3,0010ee3855fdea87602a5b7aba8e42de,2015-12-01,0.113068,40.728885,0.123086,40.565969
4,00114d74e963e47177db89bc70108537,2015-12-01,0.145440,44.266930,0.149434,44.266931
...,...,...,...,...,...,...
16091,ffef185810e44254c3a4c6395e6b4d8a,2015-12-01,0.112488,40.728885,0.162720,41.063970
16092,fffac626da707b1b5ab11e8431a4d0a2,2015-12-01,0.145047,44.444710,0.148825,44.266931
16093,fffc0cacd305dd51f316424bbb08d1bd,2015-12-01,0.151399,41.228885,0.153159,41.063970
16094,fffe4f5646aa39c7f97f95ae2679ce64,2015-12-01,0.118175,40.728885,0.127566,40.565969


In [14]:
diff['offpeak_diff_dec_january_energy'] = diff['dec_1'] - diff['price_off_peak_var']
diff['offpeak_diff_dec_january_power'] = diff['dec_2'] - diff['price_off_peak_fix']
diff = diff[['id', 'offpeak_diff_dec_january_energy','offpeak_diff_dec_january_power']]
diff.head()

,id,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000
3,0010ee3855fdea87602a5b7aba8e42de,-0.010018,0.162916
4,00114d74e963e47177db89bc70108537,-0.003994,-0.000001


In [10]:
jan_prices

,id,price_date,price_off_peak_var,price_off_peak_fix
0,0002203ffbb812588b632b9e628cc38d,2015-01-01,0.126098,40.565969
1,0004351ebdd665e6ee664792efc4fd13,2015-01-01,0.148047,44.266931
2,0010bcc39e42b3c2131ed2ce55246e3c,2015-01-01,0.150837,44.444710
3,0010ee3855fdea87602a5b7aba8e42de,2015-01-01,0.123086,40.565969
4,00114d74e963e47177db89bc70108537,2015-01-01,0.149434,44.266931
...,...,...,...,...
16091,ffef185810e44254c3a4c6395e6b4d8a,2015-01-01,0.162720,41.063970
16092,fffac626da707b1b5ab11e8431a4d0a2,2015-01-01,0.148825,44.266931
16093,fffc0cacd305dd51f316424bbb08d1bd,2015-01-01,0.153159,41.063970
16094,fffe4f5646aa39c7f97f95ae2679ce64,2015-01-01,0.127566,40.565969


In [11]:
dec_prices

,id,price_date,price_off_peak_var,price_off_peak_fix
0,0002203ffbb812588b632b9e628cc38d,2015-12-01,0.119906,40.728885
1,0004351ebdd665e6ee664792efc4fd13,2015-12-01,0.143943,44.444710
2,0010bcc39e42b3c2131ed2ce55246e3c,2015-12-01,0.201280,45.944710
3,0010ee3855fdea87602a5b7aba8e42de,2015-12-01,0.113068,40.728885
4,00114d74e963e47177db89bc70108537,2015-12-01,0.145440,44.266930
...,...,...,...,...
16091,ffef185810e44254c3a4c6395e6b4d8a,2015-12-01,0.112488,40.728885
16092,fffac626da707b1b5ab11e8431a4d0a2,2015-12-01,0.145047,44.444710
16093,fffc0cacd305dd51f316424bbb08d1bd,2015-12-01,0.151399,41.228885
16094,fffe4f5646aa39c7f97f95ae2679ce64,2015-12-01,0.118175,40.728885


In [9]:
monthly_price_by_id

,id,price_date,price_off_peak_var,price_off_peak_fix
0,0002203ffbb812588b632b9e628cc38d,2015-01-01,0.126098,40.565969
1,0002203ffbb812588b632b9e628cc38d,2015-02-01,0.126098,40.565969
2,0002203ffbb812588b632b9e628cc38d,2015-03-01,0.128067,40.728885
3,0002203ffbb812588b632b9e628cc38d,2015-04-01,0.128067,40.728885
4,0002203ffbb812588b632b9e628cc38d,2015-05-01,0.128067,40.728885
...,...,...,...,...
192997,ffff7fa066f1fb305ae285bb03bf325a,2015-08-01,0.119916,40.728885
192998,ffff7fa066f1fb305ae285bb03bf325a,2015-09-01,0.119916,40.728885
192999,ffff7fa066f1fb305ae285bb03bf325a,2015-10-01,0.119916,40.728885
193000,ffff7fa066f1fb305ae285bb03bf325a,2015-11-01,0.119916,40.728885


In [17]:
client_df = pd.read_csv("C:/Users/navir/Downloads/client_data (1).csv")

In [28]:
consumption_by_id = client_df.groupby(['id', 'date_activ', 'date_end']).agg({'imp_cons': 'mean', 'margin_gross_pow_ele': 'mean', 'net_margin':'mean', 'pow_max':'mean', }).reset_index()[client_df['churn'] ==1 ]


Checking how margin and power consumption affected the churn rate

In [29]:
consumption_by_id.head()

,id,date_activ,date_end,imp_cons,margin_gross_pow_ele,net_margin,pow_max
0,0002203ffbb812588b632b9e628cc38d,2010-01-19,2016-02-21,40.78,43.08,81.42,17.25
5,00184e957277eeef733a7b563fdabd06,2013-02-04,2016-02-04,798.16,29.76,184.06,13.20
6,001987ed9dbdab4efa274a9c7233e1f4,2004-06-10,2016-06-10,348.05,44.92,385.39,49.00
25,004eab8ff1db6804baa4742a47441dc3,2012-11-20,2016-11-20,56.37,27.03,59.60,17.32
30,0060e975b0d00bfe39fd2da5a36aefe9,2009-12-17,2016-12-06,0.00,32.64,495.70,14.50


In [30]:
sorted_cons = consumption_by_id.sort_values(by='pow_max')

In [31]:
sorted_cons

,id,date_activ,date_end,imp_cons,margin_gross_pow_ele,net_margin,pow_max
8428,94654b768198189f376d39c39f3a1d3c,2012-02-15,2016-02-15,4.17,0.00,8.61,3.300
4316,4a726466c35de4556243648a0fcb459c,2010-03-05,2016-03-05,2.00,7.04,195.86,5.196
9164,a1aa17032c549d2cb8e230acb824851d,2011-12-16,2016-12-15,141.03,9.00,61.31,5.750
12288,d77e823d24aa59beda5baa6bca37c903,2010-01-01,2016-12-31,19.05,0.00,40.62,8.000
8948,9da16ea03d4978369527d58ae9d1b2cf,2008-10-09,2016-12-31,0.00,0.00,0.00,10.000
...,...,...,...,...,...,...,...
2038,22d203ad30ae27eb51defb632d7faf2d,2013-03-12,2016-03-12,359.03,135.23,582.50,100.000
3429,3a91f9ace68e9f493855af3215347d9a,2012-05-01,2016-05-01,299.01,116.79,447.80,110.000
9855,ad958417cd4d465b1191fd69497f1cad,2008-07-01,2016-12-31,0.00,0.24,0.00,128.000
5224,5b9b8d25247edee66b7ff9ad55d2062c,2010-05-17,2016-05-17,650.38,96.12,622.29,140.000


In [34]:
sorted_cons1 = consumption_by_id.sort_values(by='imp_cons')

In [35]:
sorted_cons1

,id,date_activ,date_end,imp_cons,margin_gross_pow_ele,net_margin,pow_max
3598,3d24232ca2cf5403127a2caa0656ed4b,2012-05-10,2016-10-05,0.00,8.96,103.97,12.000
4474,4d4164ef0d2dea45d672548fdfb031a8,2010-10-23,2016-10-23,0.00,9.20,145.35,13.856
10322,b5a4bb8a74915d18acd4dfc4e42601c4,2009-12-01,2016-12-01,0.00,27.28,22.11,13.856
4497,4da86c74a4f771f9289c5d5a0056608a,2011-06-13,2016-03-26,0.00,31.64,115.41,13.856
10296,b548c8a899904cde05f88534af089561,2010-05-05,2016-05-05,0.00,24.24,45.61,13.150
...,...,...,...,...,...,...,...
6064,6a1629ea58c4a11fe7cd91d770172220,2012-10-01,2016-11-15,2287.02,4.77,803.83,73.600
6235,6d31c63c100ff939bae2a94d34479687,2013-02-15,2016-02-15,2355.71,31.20,231.48,13.856
4267,4962bc5725d812b4bd85c6c639549c6b,2008-09-17,2016-02-24,2779.70,0.12,1843.81,60.000
14345,fb7dcb0f4e0dc4ee54874eab2607c4da,2012-08-28,2016-09-01,4205.64,0.24,24570.65,150.000


In [39]:
sorted_cons1['pow_max'].max()

150.0

In [40]:
client_df['pow_max'].max()

320.0